<a href="https://colab.research.google.com/github/StarkArk/deep_learning_alphabet_soup/blob/main/alphabet_soup_optimized_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [60]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [61]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(labels=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [62]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [63]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [64]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# cut_off value --> 200
application_types_to_replace = ['T9','T13', 'T12', 'T2','T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [65]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [66]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# cutoff --> 300

# Replace in dataframe
counts = application_df['CLASSIFICATION'].value_counts()
classifications_types_to_replace = list(counts[counts<300].index)
classifications_types_to_replace

for app in classifications_types_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [67]:
# Encode categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [68]:
# Split our preprocessed data into our features and target arrays
# Use sklearn to split dataset
X = application_df_encoded.drop(["IS_SUCCESSFUL"], axis ='columns').values
y = application_df_encoded['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [69]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [70]:
num_features = len(application_df_encoded.columns) - 1
print(f'Number of Features to Model: {num_features}')

Number of Features to Model: 44


In [71]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
first_layer_dim = 44      # Input Layer
first_layer_nodes = 20
second_layer_nodes = 10
third_layer_nodes = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer_nodes, activation="relu", input_dim=first_layer_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer_nodes, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=third_layer_nodes, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 20)                900       
                                                                 
 dense_6 (Dense)             (None, 10)                210       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1121 (4.38 KB)
Trainable params: 1121 (4.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [73]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5929 - accuracy: 0.7046
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5575 - accuracy: 0.7282
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7303
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7333
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7322
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7333
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7333
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5473 - accuracy: 0.7332
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7338
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5466 - accuracy: 0.7336

In [74]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5532 - accuracy: 0.7266 - 457ms/epoch - 2ms/step
Loss: 0.5532203316688538, Accuracy: 0.7266472578048706


In [75]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Opt_1st_try.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Optimize - 2nd Attempt
- Run an optimization varying the number of layers and activations
- Add more deep layers 2 to 4
- Vary the input and deep layer activations between 'relu', 'elu', and 'tanh'
- Vary the nodes between 2 and 10


In [76]:
# Split our preprocessed data into our features and target arrays
# Use sklearn to split dataset
X = application_df_encoded.drop(["IS_SUCCESSFUL"], axis ='columns').values
y = application_df_encoded['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [77]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [78]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh', 'elu'])

    # Allow kerastuner to decide number of neurons in first layer
    units_val =hp.Int('first_units',
        min_value=10,
        max_value=18,
        step=4)
    nn_model.add(tf.keras.layers.Dense(units=units_val, activation=activation, input_dim=first_layer_dim))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 2, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=2,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [79]:
!pip install keras_tuner

In [80]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    directory="Chal21_Project0",
    hyperband_iterations=2)

In [81]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=30,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 01m 31s]
val_accuracy: 0.7273469567298889

Best val_accuracy So Far: 0.7295626997947693
Total elapsed time: 01h 20m 58s


In [82]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 18, 'num_layers': 2, 'units_0': 4, 'units_1': 4, 'units_2': 10, 'units_3': 4, 'tuner/epochs': 10, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'elu', 'first_units': 10, 'num_layers': 3, 'units_0': 6, 'units_1': 6, 'units_2': 10, 'units_3': 4, 'tuner/epochs': 30, 'tuner/initial_epoch': 10, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0049'}
{'activation': 'tanh', 'first_units': 14, 'num_layers': 3, 'units_0': 10, 'units_1': 10, 'units_2': 2, 'units_3': 8, 'tuner/epochs': 30, 'tuner/initial_epoch': 10, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0046'}


In [83]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5518 - accuracy: 0.7296 - 583ms/epoch - 2ms/step
Loss: 0.5517789125442505, Accuracy: 0.7295626997947693
268/268 - 1s - loss: 0.5538 - accuracy: 0.7292 - 652ms/epoch - 2ms/step
Loss: 0.5538442730903625, Accuracy: 0.7292128205299377
268/268 - 1s - loss: 0.5525 - accuracy: 0.7287 - 641ms/epoch - 2ms/step
Loss: 0.5524811148643494, Accuracy: 0.7287463545799255


## Optimize - 3rd Attempt
- Bring in more data and use insights from previous attempts


In [84]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [85]:
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [86]:
# There are more EIN tax numbers or Business Entities than there are Uniquely Named Charities

# len(count_ein_less2[count_ein_less2 < 2])
count_names = application_df['NAME'].value_counts()
count_names_less2 = count_names[count_names < 2]
count_names_less2 = len(count_names_less2)
count_names_more1 = len(count_names) - count_names_less2
count_names_more4 = len(count_names[count_names > 4])
count_names_more9 = len(count_names[count_names > 9])
count_names_more19 = len(count_names[count_names > 19])

print(f'Charity Names listed once: {count_names_less2} \nMore than once: {count_names_more1} \nFive times or more: {count_names_more4} \nTen times or more: {count_names_more19} \nTwenty times or more: {count_names_more19}')

Charity Names listed once: 18776 
More than once: 792 
Five times or more: 402 
Ten times or more: 121 
Twenty times or more: 121


In [87]:
# Compare this with the number of Successes
application_df['IS_SUCCESSFUL'].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [89]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `replacement_names`
# cut_off value --> 5
replacement_names = list(count_names[count_names < 2].index)

# Replace in dataframe
for name in replacement_names:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                                                 18776
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
                                                                      ...  
WASHINGTON EXPLORER SEARCH AND RESCUE                                     2
INTERNATIONAL ALLIANCE THEATRICAL STAGE EMPLOYEES & MOVING PICTURE        2
BIRTH NETWORK                                                             2
NATIONAL ASSOCIATON OF CORPORATE DIRECTORS                                2
NATIONAL SOCIETY COLONIAL DAMES XVII CENTURY                              2
Name: NAME, Length: 793, dtype: int64

In [90]:
# Drop non-beneficial 'EIN' column.
application_df = application_df.drop(labels=['EIN'], axis=1)
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,Other,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,Other,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [91]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [92]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# cut_off value --> 200
application_types_to_replace = ['T9', 'T13', 'T12', 'T2','T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [93]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [94]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# cut-off --> 100
counts = application_df['CLASSIFICATION'].value_counts()
classifications_types_to_replace = list(counts[counts<100].index)
classifications_types_to_replace

# Replace in dataframe
for app in classifications_types_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [95]:
# Encode categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_4-H CLUBS & AFFILIATED 4-H ORGANIZATIONS,NAME_AACE INTERNATIONAL,NAME_ACADEMY OF GENERAL DENTISTRY,NAME_ACADEMY OF MANAGED CARE PHARMACY,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTIVE 20-30 UNITED STATES AND CANADA,NAME_ACTS MINISTRY,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [96]:
# Split our preprocessed data into our features and target arrays
# Use sklearn to split dataset
X = application_df_encoded.drop(["IS_SUCCESSFUL"], axis ='columns').values
y = application_df_encoded['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [97]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [98]:
num_features = len(application_df_encoded.columns) - 1
print(f'Number of Features to Model: {num_features}')

Number of Features to Model: 842


In [99]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
first_layer_nodes = 842      # Input Layer
second_layer_nodes = 80
third_layer_nodes = 40
fourth_layer_nodes = 20
output_layer_nodes = 1

nn3 = tf.keras.models.Sequential()

# Input layer
nn3.add(tf.keras.layers.Dense(units=first_layer_nodes, activation="relu", input_dim=first_layer_nodes))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=second_layer_nodes, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=third_layer_nodes, activation="relu"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=fourth_layer_nodes, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=output_layer_nodes, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 842)               709806    
                                                                 
 dense_6 (Dense)             (None, 80)                67440     
                                                                 
 dense_7 (Dense)             (None, 40)                3240      
                                                                 
 dense_8 (Dense)             (None, 20)                820       
                                                                 
 dense_9 (Dense)             (None, 1)                 21        
                                                                 
Total params: 781327 (2.98 MB)
Trainable params: 781327 (2.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [100]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [101]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30
804/804 [==============================] - 11s 13ms/step - loss: 0.4631 - accuracy: 0.7799
Epoch 2/30
804/804 [==============================] - 10s 12ms/step - loss: 0.4202 - accuracy: 0.8035
Epoch 3/30
804/804 [==============================] - 9s 11ms/step - loss: 0.4118 - accuracy: 0.8061
Epoch 4/30
804/804 [==============================] - 12s 15ms/step - loss: 0.4085 - accuracy: 0.8072
Epoch 5/30
804/804 [==============================] - 12s 15ms/step - loss: 0.4078 - accuracy: 0.8089
Epoch 6/30
804/804 [==============================] - 11s 14ms/step - loss: 0.4059 - accuracy: 0.8075
Epoch 7/30
804/804 [==============================] - 11s 14ms/step - loss: 0.4051 - accuracy: 0.8091
Epoch 8/30
804/804 [==============================] - 10s 13ms/step - loss: 0.4029 - accuracy: 0.8106
Epoch 9/30
804/804 [==============================] - 9s 11ms/step - loss: 0.4023 - accuracy: 0.8111
Epoch 10/30
804/804 [==============================] - 10s 13ms/step - loss: 0.4016 

In [102]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4826 - accuracy: 0.8020 - 1s/epoch - 4ms/step
Loss: 0.4825701415538788, Accuracy: 0.8019825220108032


In [103]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Opt_3rd_try.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
